# CS 282
### Programming Assignment 1
#### Item 2

Jan Lendl R. Uy

2019-00312

#### Sytem Specifications
- macOS Sequoia 15.0.1
- Macbook Air M1 (ARM), 8-Core CPU and 8-Core GPU

#### Notes when Running the Notebook
- Video Player Controls
  - Use the 'Position' slider to navigate the video (0-10)
  - Click the PAUSE/PLAY button at the bottom pane to control playback
  - Press 'q' or 'ESC' to terminate OpenCV2 windows.
- OpenCV2 windows tend to be buggy when attempted to be closed. They are best terminated through the following:
  - For macOS: Click 'Force Quit' in Activity Monitor
  - For Windows: Click 'End Task' in Task Manager

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import cv2
import os

In [3]:
class VideoPlayer:
    def __init__(self, video_path):
        """Initialize the video player with the specified video file."""
        # Store video path
        self.video_path = video_path
        
        # Check if file exists
        if not os.path.isfile(self.video_path):
            raise FileNotFoundError(f"Error: File '{self.video_path}' not found")
        
        # Open video
        self.cap = cv2.VideoCapture(self.video_path)
        if not self.cap.isOpened():
            raise IOError(f"Error: Could not open video file '{self.video_path}'")
        
        # Get video properties
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        if self.fps <= 0:
            self.fps = 30  # Default if invalid
        
        # State variables
        self.paused = False
        self.current_pos = 0
        self.fix_orientation = False
        
        # UI positions for tracking
        self.current_button_x = 20
        self.current_button_y = self.height + 10
        self.current_button_width = 100
        self.current_button_height = 40
        
        # Window setup
        self.window_name = 'Video Player'
        cv2.namedWindow(self.window_name)
        
        # Set up callbacks
        cv2.setMouseCallback(self.window_name, self.mouse_callback)
        
    def setup_trackbar(self):
        """Create the position slider."""
        cv2.createTrackbar('Position', self.window_name, 0, 10, self.on_slider_change)
        
    def display_frame_with_controls(self, frame):
        """Display the current frame with UI controls."""
        # Fix orientation if needed
        if self.fix_orientation:
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        # Get dimensions after rotation
        frame_height, frame_width = frame.shape[:2]
        
        # Create a larger canvas with space for controls
        controls_height = 60
        
        # Minor use of numpy for creation of a canvas
        import numpy as np
        canvas = np.ones((frame_height + controls_height, frame_width, 3), dtype=np.uint8) * 240  # Light gray
        
        # Add video frame to canvas
        canvas[0:frame_height, 0:frame_width] = frame
        
        # Draw separator line
        cv2.line(canvas, (0, frame_height), (frame_width, frame_height), (180, 180, 180), 2)
        
        # Button dimensions
        button_width = 100
        button_height = 40
        button_x = 20
        button_y = frame_height + 10
        
        # Draw pause/play button
        label = "PAUSE" if not self.paused else "PLAY"
        button_color = (120, 120, 120) if self.paused else (0, 120, 0)  # Gray when paused, green when playing
        
        cv2.rectangle(canvas, 
                     (button_x, button_y), 
                     (button_x + button_width, button_y + button_height), 
                     button_color, 
                     -1)  # Filled rectangle
        
        # Add button text
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        text_x = button_x + (button_width - text_size[0]) // 2
        text_y = button_y + (button_height + text_size[1]) // 2
        cv2.putText(canvas, label, (text_x, text_y), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Add slider label
        cv2.putText(canvas, "Position:", (button_x + button_width + 20, button_y + 25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        
        # Display the combined canvas
        cv2.imshow(self.window_name, canvas)
        
        # Update button position tracking
        self.current_button_x = button_x
        self.current_button_y = button_y
        self.current_button_width = button_width
        self.current_button_height = button_height
    
    def on_slider_change(self, pos):
        """Handle slider position changes."""
        frame_pos = int((pos / 10.0) * self.total_frames)
        if frame_pos >= self.total_frames:
            frame_pos = self.total_frames - 1
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        ret, frame = self.cap.read()
        if ret:
            self.display_frame_with_controls(frame)
    
    def mouse_callback(self, event, x, y, flags, param):
        """Handle mouse events for UI interaction."""
        if event == cv2.EVENT_LBUTTONDOWN:
            # Check if click is within button area
            if (self.current_button_x <= x <= self.current_button_x + self.current_button_width and 
                self.current_button_y <= y <= self.current_button_y + self.current_button_height):
                # Toggle pause state
                self.toggle_pause()
    
    def toggle_pause(self):
        """Toggle the pause state and update display."""
        self.paused = not self.paused
        print("Video", "paused" if self.paused else "playing")
        
        # Update the display
        current_frame = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
        # Go back one frame to stay at current position
        if current_frame > 0:
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame - 1)
        ret, frame = self.cap.read()
        if ret:
            self.display_frame_with_controls(frame)
    
    def ask_orientation(self):
        """Ask user if they want to fix video orientation."""
        print(f"Video dimensions: {self.width}x{self.height}")
        response = input("Fix orientation by rotating 90 degrees clockwise? (y/n): ").lower()
        self.fix_orientation = response.startswith('y')
        
        # Update button position if orientation is fixed
        if self.fix_orientation:
            self.current_button_y = self.width + 10
    
    def run(self):
        """Run the video player."""
        # Ask about orientation
        self.ask_orientation()
        
        # Create slider
        self.setup_trackbar()
        
        # Read first frame
        ret, frame = self.cap.read()
        if not ret:
            print("Error reading the first frame")
            return
        
        # Show first frame with controls
        self.display_frame_with_controls(frame)
        
        # Print instructions
        print("\nControls:")
        print("- Use the 'Position' slider to navigate the video (0-10)")
        print("- Click the PAUSE/PLAY button to control playback")
        print("- Press 'q' or 'ESC' to quit")
        
        # Main loop
        while True:
            # Handle key presses (5ms wait for responsiveness)
            key = cv2.waitKey(5) & 0xFF
            if key == 27 or key == ord('q'):
                break
            
            # Space bar can also toggle pause
            if key == 32:  # Space key
                self.toggle_pause()
            
            # If not paused, play video
            if not self.paused:
                ret, frame = self.cap.read()
                
                # If reached end of video, loop back
                if not ret:
                    self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                    ret, frame = self.cap.read()
                    self.current_pos = 0
                
                if ret:
                    # Show the frame with controls
                    self.display_frame_with_controls(frame)
                    
                    # Update position
                    current_frame = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
                    new_pos = int((current_frame / self.total_frames) * 10)
                    
                    # Only update trackbar if position changed
                    if new_pos != self.current_pos:
                        self.current_pos = new_pos
                        cv2.setTrackbarPos('Position', self.window_name, self.current_pos)
                    
                    # Control playback speed
                    cv2.waitKey(int(1000/self.fps))
        
        # Clean up
        self.clean_up()
    
    def clean_up(self):
        """Release resources and close windows."""
        if hasattr(self, 'cap') and self.cap.isOpened():
            self.cap.release()
        cv2.destroyAllWindows()
        # Force window closing in Jupyter
        for i in range(5):
            cv2.waitKey(1)
    
    def __del__(self):
        """Destructor to ensure resources are properly released."""
        self.clean_up()

In [4]:
# Path to sample videos (uncomment desired video to be played)
# video_path = "videos/central.MOV"
# video_path = "videos/merlion.MOV"
video_path = input("Enter the path to the video: ")

# Create and run video player
player = VideoPlayer(video_path)
player.run()

Video dimensions: 1920x1080

Controls:
- Use the 'Position' slider to navigate the video (0-10)
- Click the PAUSE/PLAY button to control playback
- Press 'q' or 'ESC' to quit


2025-03-02 14:37:12.355 Python[56516:21731213] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-02 14:37:12.355 Python[56516:21731213] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Video paused
Video playing
Video paused
Video playing
Video paused
